In [13]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict

In [14]:
df = pd.DataFrame()
for chunk in pd.read_csv('./news_dataset.csv', usecols=['content'], chunksize=1000):
    df = pd.concat([df, chunk], ignore_index=True)

In [15]:
df.head()

,content
0,WASHINGTON — Congressional Republicans have...
1,"After the bullet shells get counted, the blood..."
2,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,"Death may be the great equalizer, but it isn’t..."
4,"SEOUL, South Korea — North Korea’s leader, ..."


In [16]:
import re
df["content"] = df["content"].str.lower() 

# df["content"] = df["content"].str.translate(None, ",!.; -@!%^&*)(\\\'\"")
df['content'] = df['content'].map(lambda x: re.sub(r'[,!.;+-@!%^&*)(\\\'\"“”’—]', '', str(x)))
df['content'] = df['content'].map(lambda x: re.sub(r'\W+', ' ', str(x)))

In [17]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words("english")
df['content'] = df['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Quentindh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
df["content"]


0         washington congressional republicans new fear ...
1         bullet shells get counted blood dries votive c...
2         walt disneys bambi opened critics praised spar...
3         death may great equalizer isnt necessarily eve...
4         seoul south korea north koreas leader kim said...
                                ...                        
141580    uber driver keith avila picked passenger looke...
141581    crews friday continued search lake erie plane ...
141582    obama administration announced massive effort ...
141583    story updated new law michigan prohibit local ...
141584    nations first recreational marijuana shop open...
Name: content, Length: 141585, dtype: object

In [19]:
nr_topics = 10
topics = list(range(nr_topics))
topics

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
doc_topics = []
word_topics = defaultdict(lambda: defaultdict(int))

assigned = []
for index, row in df.iterrows():
    if index%1000 == 0:
        print(index)
        if index == 2000:
            break
    t = defaultdict(int)
    assigned_doc = []
    for word in row["content"].split():
        word_topic = random.sample(topics, 1)[0]
        word_topics[word][word_topic] += 1
        assigned_doc.append(word_topic)
        t[word_topic] += 1
    assigned.append(assigned_doc)
    doc_topics.append(t)
        

0


In [ ]:
doc_topics

In [ ]:
word_topics

In [ ]:
assigned[0]

In [ ]:
alpha = 10
gamma = 4

def sum_doc(doc_index):
    return len(assigned[doc_index]) + nr_topics*alpha

def sum_topic(topic):
    s = 0
    for doc in doc_topics:
        s += doc[topic] + gamma
        
    return s
def alltopics():
    lis = []
    
    for t in topics:
        lis.append(sum_topic(t))
    
    return lis

sum_topics = alltopics()
for index, row in df.iterrows():
    # print(index)
    for i, word in enumerate(row["content"].split()):
        t = assigned[index][i]
        word_topics[word][t] -= 1
        doc_topics[index][t] -= 1
        sum_topics[t] -= 1
        
        topic_scores = []
        for top in range(nr_topics):
            n_dk = doc_topics[index][top]
            v_kwdn = word_topics[word][top]

            s1 = sum_doc(index)
            s2 = sum_topics[top]
            topic_scores.append(((n_dk + alpha)/s1) * ((v_kwdn + gamma)/s2))
        
        z_new = random.choices(topics, weights=topic_scores, k=1)[0]
        word_topics[word][z_new] += 1
        doc_topics[index][z_new] += 1
        sum_topics[z_new] += 1
        assigned[index][i] = z_new
        